In [1]:
import nltk
from collections import defaultdict, Counter
import gensim, logging
import nltk
from gensim.models.keyedvectors import KeyedVectors
from gensim.similarities.index import AnnoyIndexer
from multiprocessing import Process, Pool
import os
import collections
from random import shuffle
from copy import deepcopy
from collections import OrderedDict

In [2]:
# nltk.corpus.gutenberg.fileids()
# total_corpus = []
# for file in nltk.corpus.gutenberg.fileids():
#     with open((nltk.corpus.gutenberg.words(file).fileid.path), 'r') as f:
#         for line in f.readlines():
#             try:
#                 total_corpus.append(line.split())
#             except:
#                 print ("hi")
#                 print (line)

# with open(emma.fileid.path) as f:
#     for line in f.readlines():
#         print (line)

# emma = nltk.corpus.gutenberg.words('austen-emma.txt')
# vocab = list(set(emma))
# len(vocab)

In [3]:
def extract_patterns_in_words(patterns,pattern_counter,word1,word2,max_len):
    i = 1
    while(word1[:i] == word2[:i]):
        i = i + 1
    if i != 1 and i > max(len(word1[i-1:]), len(word2[i-1:])) < max_len:
        pattern_counter[("suffix",word1[i-1:], word2[i-1:])] += 1
        if ("suffix",word1[i-1:], word2[i-1:]) in patterns:
            patterns[("suffix",word1[i-1:], word2[i-1:])].append((word1, word2))
        else:
            patterns[("suffix",word1[i-1:], word2[i-1:])] = [(word1, word2)]
#         patterns[("suffix",word1[i-1:], word2[i-1:], word1, word2)] += 1
    i = 1
    while(word1[-i:] == word2[-i:]):
        i = i + 1
    if i != 1 and max(len(word1[:-i+1]), len(word2[:-i+1])) < max_len:
        pattern_counter[("prefix",word1[:-i+1], word2[:-i+1])] += 1
        if ("prefix",word1[:-i+1], word2[:-i+1]) in patterns:
            patterns[("prefix",word1[:-i+1], word2[:-i+1])].append((word1, word2))
        else:
            patterns[("prefix",word1[:-i+1], word2[:-i+1])] = [(word1, word2)]
#         patterns[("prefix",word1[:-i+1], word2[:-i+1], word1, word2)] += 1
    return patterns

In [4]:
def build_pattern_dict(vocab,max_len = 6):
    patterns  = defaultdict(list)
#     print (patterns)
    pattern_counter = Counter()
    for word in vocab:
        for second_word in vocab:
            if word != second_word:
                extract_patterns_in_words(patterns,pattern_counter,word,second_word,max_len)
    return patterns, pattern_counter

In [5]:
def downsample_patterns():
    #Downsample to include only top 1000
    pattern_1000 = defaultdict(list)
    for pattern,items in patterns.items():
        shuffle(items)
        pattern_1000[pattern] = items[:1000]
    return pattern_1000

In [6]:
def pair_wise_similarity(word_pair1, word_pair2,annoy_index=None, topn = 10,):
    closest_n = word_vectors.most_similar(positive=[word_pair2[0], word_pair1[1]], negative=[word_pair1[0]], topn=topn)
#     print (word_pair2[1])
#     print (closest_n)
    for word, cos_sim in closest_n:
        if word == word_pair2[1]:
            return True
    return False

def annoy_pair_wise_similarity(word_pair1, word_pair2,annoy_index, topn = 10):
    closest_n = word_vectors.most_similar(positive=[word_pair2[0], word_pair1[1]], negative=[word_pair1[0]], topn=topn, indexer=annoy_index)
#     print (word_pair2[1])
#     print (closest_n)
    for word, cos_sim in closest_n:
        if word == word_pair2[1]:
            return True
    return False
    

In [7]:
def index_vector(word_vectors, dimensions=300):
    fname = '../data/annoy.index'
    # Persist index to disk
    if os.path.exists(fname):
        annoy_index = AnnoyIndexer()
        annoy_index.load(fname)
        annoy_index.model = word_vectors
    else:
        annoy_index = AnnoyIndexer(word_vectors, dimensions)
        annoy_index.save(fname)
    return annoy_index

In [8]:
def calculate_hit_rate(patterns, similarity_function):
    hit_rates_rules = {}
    for (pattern,support_set) in patterns.items():
        hit_rates_word_pair = {}
        for pair1 in support_set:
            hit_count = 0
            hit_pairs = set()
            for pair2 in support_set:
                if pair1 != pair2 and similarity_function(pair1, pair2, 10):
                    hit_count += 1
                    hit_pairs.add(pair2)
            if len(support_set) ==1:
                total = 1
            else:
                total = len(support_set) - 1
            if hit_count != 0:
                hit_rates_word_pair[pair1] =  hit_pairs
        if len(support_set) != 1 and hit_rates_word_pair:
            hit_rates_rules[pattern] = hit_rates_word_pair
    return hit_rates_rules

In [9]:
#pattern_counter.most_common()[:-20:-1]

In [10]:
%time word_vectors = KeyedVectors.load_word2vec_format('/home/raja/GoogleNews-vectors-negative300.bin.gz', binary=True, limit=5000)

CPU times: user 132 ms, sys: 0 ns, total: 132 ms
Wall time: 134 ms


In [11]:
%time patterns, pattern_counter = build_pattern_dict(word_vectors.vocab.keys())

CPU times: user 20.7 s, sys: 356 ms, total: 21.1 s
Wall time: 21.1 s


In [12]:
pattern_counter.most_common(20)

[(('suffix', '', 's'), 598),
 (('suffix', 's', ''), 598),
 (('suffix', 'ed', ''), 168),
 (('suffix', '', 'ed'), 168),
 (('suffix', 'ing', ''), 149),
 (('suffix', '', 'ing'), 149),
 (('suffix', 'ing', 'ed'), 141),
 (('suffix', 'ed', 'ing'), 141),
 (('suffix', '', 'd'), 127),
 (('suffix', 'd', ''), 127),
 (('suffix', 'ed', 's'), 86),
 (('suffix', 's', 'ed'), 86),
 (('suffix', '', 'ly'), 80),
 (('suffix', 'ly', ''), 80),
 (('suffix', 's', 'ing'), 76),
 (('suffix', 'ing', 's'), 76),
 (('suffix', 'ing', 'e'), 74),
 (('suffix', 'e', 'ing'), 74),
 (('suffix', 'd', 's'), 62),
 (('suffix', 's', 'd'), 62)]

In [13]:
%time sampled_patterns = downsample_patterns()

CPU times: user 2.84 s, sys: 68 ms, total: 2.91 s
Wall time: 2.9 s


In [14]:
# word_vectors_annoy = deepcopy(word_vectors)
# word_vectors.init_sims(replace=True)
# %time annoy_index = index_vector(word_vectors=word_vectors, dimensions=300)
# %time hit_rates = calculate_hit_rate(sampled_patterns, annoy_pair_wise_similarity, annoy_index)

In [15]:
#pair_wise_similarity(("asking", "banking"), ("ask", "bank"))

In [16]:
%time hit_rates = calculate_hit_rate(sampled_patterns, pair_wise_similarity)

CPU times: user 16min 55s, sys: 4.7 s, total: 17min
Wall time: 4min 15s


In [17]:
#%time hit_rates = calculate_hit_rate(sampled_patterns, annoy_pair_wise_similarity, annoy_index)

In [18]:
sampled_patterns[('suffix', '', 's')]

[('program', 'programs'),
 ('doctor', 'doctors'),
 ('artist', 'artists'),
 ('stand', 'stands'),
 ('practice', 'practices'),
 ('ball', 'balls'),
 ('highlight', 'highlights'),
 ('role', 'roles'),
 ('water', 'waters'),
 ('seem', 'seems'),
 ('worker', 'workers'),
 ('back', 'backs'),
 ('raise', 'raises'),
 ('star', 'stars'),
 ('explain', 'explains'),
 ('subject', 'subjects'),
 ('right', 'rights'),
 ('college', 'colleges'),
 ('return', 'returns'),
 ('effort', 'efforts'),
 ('holiday', 'holidays'),
 ('playoff', 'playoffs'),
 ('result', 'results'),
 ('model', 'models'),
 ('walk', 'walks'),
 ('Democrat', 'Democrats'),
 ('strike', 'strikes'),
 ('turn', 'turns'),
 ('spot', 'spots'),
 ('procedure', 'procedures'),
 ('election', 'elections'),
 ('add', 'adds'),
 ('set', 'sets'),
 ('period', 'periods'),
 ('list', 'lists'),
 ('fund', 'funds'),
 ('fee', 'fees'),
 ('file', 'files'),
 ('contribution', 'contributions'),
 ('hand', 'hands'),
 ('bond', 'bonds'),
 ('requirement', 'requirements'),
 ('road', 'roa

In [19]:
patterns[('suffix', '', 's')]

[('program', 'programs'),
 ('doctor', 'doctors'),
 ('artist', 'artists'),
 ('stand', 'stands'),
 ('practice', 'practices'),
 ('ball', 'balls'),
 ('highlight', 'highlights'),
 ('role', 'roles'),
 ('water', 'waters'),
 ('seem', 'seems'),
 ('worker', 'workers'),
 ('back', 'backs'),
 ('raise', 'raises'),
 ('star', 'stars'),
 ('explain', 'explains'),
 ('subject', 'subjects'),
 ('right', 'rights'),
 ('college', 'colleges'),
 ('return', 'returns'),
 ('effort', 'efforts'),
 ('holiday', 'holidays'),
 ('playoff', 'playoffs'),
 ('result', 'results'),
 ('model', 'models'),
 ('walk', 'walks'),
 ('Democrat', 'Democrats'),
 ('strike', 'strikes'),
 ('turn', 'turns'),
 ('spot', 'spots'),
 ('procedure', 'procedures'),
 ('election', 'elections'),
 ('add', 'adds'),
 ('set', 'sets'),
 ('period', 'periods'),
 ('list', 'lists'),
 ('fund', 'funds'),
 ('fee', 'fees'),
 ('file', 'files'),
 ('contribution', 'contributions'),
 ('hand', 'hands'),
 ('bond', 'bonds'),
 ('requirement', 'requirements'),
 ('road', 'roa

In [ ]:
# def temp(pattern, transformations):
# #     ll = [transformations[transformation] for transformation in transformations]
# #     print (ll)
# #     print (len(ll), len(transformations))
#     print (transformations)
#     prototypes = []
#     support_set = set(sampled_patterns[pattern])
# #     print (pattern)
# #     print (support_set)
#     while True:
#         explains_by_count = sorted(transformations.items(), key=lambda kv: -len(kv[1]))
#         print (explains_by_count)
#         best = explains_by_count[0]
#         if len(best[1]) >= 10: #The prototype explains more than 10 word pairs
#             prototypes.append((best[0][0], len(best[1]) / float(len(support_set))))
#         else:
#             break
#         del transformations[best[0]]
#         #Remove all explained pairs from the support set
#         support_set = support_set - best[1]
#         for k, v in transformations.items():
#             transformations[k] = transformations[k] - best[1]
#         explains_by_count.pop(0)

#         if not (len(support_set) >= 10 and len(explains_by_count) and explains_by_count[0] >= 10):
#             break
#     print (prototypes)

In [20]:
cons = 5
def get_morpho_rules(patterns):
    global morphological_rules
    for pattern in patterns:
        transformations = patterns[pattern]
#         ll = [transformations[transformation] for transformation in transformations]
    #     print (ll)
    #     print (len(ll), len(transformations))
        support_set = set(sampled_patterns[pattern])
    #     directions = []
        while(True):
            transformations_by_count = sorted(transformations.items(), key=lambda kv: len(kv[1]), reverse=True)
            best = transformations_by_count[0]
    #         print (transformations_by_count)
    #         print (transformations)
            if len(best[1]) >= cons:
                morphological_rules[best[0]] = (pattern, len(best[1]) / float(len(support_set)),  best[1])
    #             directions.append(best)
                del transformations[best[0]]
            else:
                break

            #Remove all explained pairs from the support set
            #TODO: Remove best[0] from support set and transformations
            support_set = support_set - best[1]
            for k, v in transformations.items():
    #             print ("*"*50)
    #             print (transformations[k])
                transformations[k] = transformations[k] - best[1]
    #             print (transformations[k])
    #             print ("__"*50)

            transformations_by_count.pop(0)
            if not (len(support_set) >= cons and len(transformations_by_count) and len(transformations_by_count[0][1]) >= cons):
                break
    #     return directions
    #         print (best_transformation,best_support)


In [22]:
#%time patterns = calculate_hit_rate(sampled_patterns, pair_wise_similarity)
morphological_rules = {}
%time get_morpho_rules(hit_rates)
        



CPU times: user 192 ms, sys: 0 ns, total: 192 ms
Wall time: 193 ms


In [23]:
len(morphological_rules)

156

In [24]:
for item in morphological_rules:
    print (item, morphological_rules[item])
    print ("*" * 50)

('appointment', 'appointed') (('suffix', 'ment', 'ed'), 0.875, {('establishment', 'established'), ('employment', 'employed'), ('development', 'developed'), ('treatment', 'treated'), ('argument', 'argued'), ('deployment', 'deployed'), ('investment', 'invested')})
**************************************************
('defending', 'defender') (('suffix', 'ing', 'er'), 0.8, {('reporting', 'reporter'), ('opening', 'opener'), ('producing', 'producer'), ('manufacturing', 'manufacturer'), ('developing', 'developer'), ('writing', 'writer'), ('making', 'maker'), ('singing', 'singer'), ('teaching', 'teacher'), ('providing', 'provider'), ('starting', 'starter'), ('receiving', 'receiver'), ('playing', 'player'), ('driving', 'driver'), ('managing', 'manager'), ('striking', 'striker'), ('leading', 'leader'), ('winning', 'winner'), ('working', 'worker'), ('pitching', 'pitcher')})
**************************************************
('driving', 'drivers') (('suffix', 'ing', 'ers'), 0.84, {('voting', 'voter